### LSTM 을 활용한 주가 예측 모델
##### 대표적인 보조지표만 추가

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import datetime as dt


%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
import sqlite3
con = sqlite3.connect(r"C:\Users\구남이\OneDrive\바탕 화면\활동들\한이음 멘토링\stock.db")
df = pd.read_sql("SELECT * FROM '039490'", con, index_col=None)
df.rename(columns={'index':'date'}, inplace=True)
df = df.set_index('date')
df.index = pd.DatetimeIndex(df.index)
df.sort_index(ascending=True, inplace=True)
df

,open,high,low,close,volume
date,,,,,
2004-04-23,6076,6799,5353,5353,1415109
2004-04-26,5436,5741,5261,5398,343283
2004-04-27,5452,5482,5033,5063,299489
2004-04-28,5071,5452,4995,5444,182604
2004-04-29,5117,5436,5025,5101,109911
...,...,...,...,...,...
2022-05-11,87500,87500,85500,85800,108071
2022-05-12,84700,85300,82700,83200,104007
2022-05-13,84000,87400,83500,85800,86525


In [3]:
# 이동평균선 추가
ma = [5,20,60,120]
for days in ma:
    df['ma_'+str(days)] = df['close'].rolling(window = days).mean()

df.dropna(inplace=True)
df 

,open,high,low,close,volume,ma_5,ma_20,ma_60,ma_120
date,,,,,,,,,
2004-10-14,3959,4066,3906,4035,171655,3940.8,3963.60,3421.083333,3346.366667
2004-10-15,4066,4096,3959,3990,128530,3959.2,3946.85,3442.283333,3335.008333
2004-10-18,3997,3997,3860,3868,77233,3944.0,3935.05,3461.383333,3322.258333
2004-10-19,3868,3914,3830,3853,63272,3935.0,3922.90,3481.183333,3312.175000
2004-10-20,3807,3853,3769,3769,118227,3903.0,3909.60,3499.450000,3298.216667
...,...,...,...,...,...,...,...,...,...
2022-05-11,87500,87500,85500,85800,108071,89460.0,94620.00,97348.333333,100015.833333
2022-05-12,84700,85300,82700,83200,104007,87480.0,93785.00,97120.000000,99788.333333
2022-05-13,84000,87400,83500,85800,86525,86360.0,93050.00,96945.000000,99603.333333


In [4]:
! pip install ta
import ta

H, L, C, V = df['high'], df['low'], df['close'], df['volume']

## 채널지표
df['Parabolic SAR'] = ta.trend.psar_down(
    high=H, low=L, close=C, fillna=True)
## 추세지표
df['MACD'] = ta.trend.macd(close=C, fillna=True)
## 시장강도지표
df['ROC'] = ta.momentum.roc(close=C, fillna=True)
df['MFI'] = ta.volume.money_flow_index(
    high=H, low=L, close=C, volume=V, fillna=True)
df['NVI'] = ta.volume.negative_volume_index(close=C, volume=V, fillna=True)
df['OBV'] = ta.volume.on_balance_volume(close=C, volume=V, fillna=True)
df['CCI'] = ta.trend.cci(high=H, low=L, close=C, fillna=True)
## 모멘텀지표
df['ADX'] = ta.trend.adx(high=H, low=L, close=C, fillna=True)
df['ADI'] = ta.volume.acc_dist_index(
    high=H, low=L, close=C, volume=V, fillna=True)
df['SRSI'] = ta.momentum.stochrsi(close=C, fillna=True)
df['SR'] = ta.momentum.stoch(close=C, high=H, low=L, fillna=True)
df['RSI'] = ta.momentum.rsi(close=C, fillna=True)
df['TSI'] = ta.momentum.tsi(close=C, fillna=True)
df['TRIX'] = ta.trend.trix(close=C, fillna=True)
df['WR'] = ta.momentum.williams_r(high=H, low=L, close=C, fillna=True)

df


,open,high,low,close,volume,ma_5,ma_20,ma_60,ma_120,Parabolic SAR,...,OBV,CCI,ADX,ADI,SRSI,SR,RSI,TSI,TRIX,WR
date,,,,,,,,,,,,,,,,,,,,,
2004-10-14,3959,4066,3906,4035,171655,3940.8,3963.60,3421.083333,3346.366667,4066.000000,...,171655,0.000000,0.000000,1.051387e+05,0.000000,80.625000,100.000000,0.000000,-93.381081,-19.375000
2004-10-15,4066,4096,3959,3990,128530,3959.2,3946.85,3442.283333,3335.008333,4066.000000,...,43125,66.666667,0.000000,3.477555e+04,0.000000,44.210526,0.000000,-100.000000,-0.002178,-55.789474
2004-10-18,3997,3997,3860,3868,77233,3944.0,3935.05,3461.383333,3322.258333,4066.000000,...,-34108,-100.000000,0.000000,-3.343754e+04,0.000000,3.389831,0.000000,-100.000000,-0.011623,-96.610169
2004-10-19,3868,3914,3830,3853,63272,3935.0,3922.90,3481.183333,3312.175000,4096.000000,...,-97380,-90.045662,0.000000,-6.206059e+04,0.000000,8.646617,0.000000,-100.000000,-0.026237,-91.353383
2004-10-20,3807,3853,3769,3769,118227,3903.0,3909.60,3499.450000,3298.216667,4085.360000,...,-215607,-110.500611,0.000000,-1.802876e+05,0.000000,0.000000,0.000000,-100.000000,-0.047711,-100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-11,87500,87500,85500,85800,108071,89460.0,94620.00,97348.333333,100015.833333,92289.505060,...,49229575,-189.708664,24.016382,1.167620e+07,0.000000,2.173913,21.028717,-35.840057,-0.233989,-97.826087
2022-05-12,84700,85300,82700,83200,104007,87480.0,93785.00,97120.000000,99788.333333,90931.604048,...,49125568,-209.031368,26.812631,1.161220e+07,0.000000,3.311258,17.870117,-41.172548,-0.275020,-96.688742
2022-05-13,84000,87400,83500,85800,86525,86360.0,93050.00,96945.000000,99603.333333,89285.283238,...,49212093,-147.749159,28.108887,1.162773e+07,0.465648,23.134328,29.305533,-41.365973,-0.313561,-76.865672


In [5]:
from sklearn.preprocessing import MinMaxScaler

# 피처값 스케일링
scaler = MinMaxScaler()

scaled_df = scaler.fit_transform(df.drop('close', axis=1))
scaled_df = pd.DataFrame(scaled_df, columns = df.drop('close', axis=1).columns)

In [6]:
# 타켓 스케일링
scaler1 = MinMaxScaler()
scaled_df['close'] = scaler1.fit_transform(df['close'].values.reshape(-1,1))

In [7]:
scaled_df

,open,high,low,volume,ma_5,ma_20,ma_60,ma_120,Parabolic SAR,MACD,...,CCI,ADX,ADI,SRSI,SR,RSI,TSI,TRIX,WR,close
0,0.003777,0.004016,0.003642,0.119625,0.003413,0.003428,0.000000,0.000860,0.003225,0.443445,...,0.425027,0.000000,0.294518,0.000000,0.806250,1.000000,0.565490,0.000000,0.806250,0.004173
1,0.004449,0.004199,0.003990,0.089071,0.003534,0.003313,0.000154,0.000774,0.003225,0.443213,...,0.509405,0.000000,0.291081,0.000000,0.442105,0.000000,0.000000,0.977135,0.442105,0.003890
2,0.004015,0.003596,0.003341,0.052727,0.003434,0.003232,0.000294,0.000677,0.003225,0.442403,...,0.298460,0.000000,0.287749,0.000000,0.033898,0.000000,0.000000,0.977036,0.033898,0.003121
3,0.003205,0.003090,0.003144,0.042836,0.003374,0.003148,0.000438,0.000601,0.003408,0.441703,...,0.311059,0.000000,0.286351,0.000000,0.086466,0.000000,0.000000,0.976883,0.086466,0.003026
4,0.002821,0.002718,0.002745,0.081771,0.003163,0.003057,0.000571,0.000495,0.003343,0.440740,...,0.285169,0.000000,0.280576,0.000000,0.000000,0.000000,0.000000,0.976659,0.000000,0.002496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4347,0.528723,0.512472,0.538159,0.074576,0.567238,0.626130,0.684249,0.734314,0.541295,0.277809,...,0.184918,0.361673,0.859703,0.000000,0.021739,0.210287,0.362818,0.974709,0.021739,0.519628
4348,0.511128,0.499065,0.519817,0.071697,0.554184,0.620395,0.682586,0.732588,0.533014,0.246397,...,0.160462,0.403783,0.856577,0.000000,0.033113,0.178701,0.332663,0.974280,0.033113,0.503237
4349,0.506730,0.511862,0.525057,0.059311,0.546800,0.615346,0.681311,0.731184,0.522973,0.237469,...,0.238025,0.423304,0.857336,0.465648,0.231343,0.293055,0.331569,0.973877,0.231343,0.519628
4350,0.524953,0.513081,0.539469,0.037247,0.542712,0.611156,0.679647,0.729913,0.514940,0.233338,...,0.283793,0.439666,0.855200,0.482154,0.250000,0.297109,0.331549,0.973515,0.250000,0.520258


In [8]:
#train,test 분할
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(scaled_df.drop('close', 1), scaled_df['close'], test_size=0.3, random_state=0, shuffle=False)
    

In [9]:
# tensorflow dataset을 활용한 시퀀스 데이터셋 구성
import tensorflow as tf
def windowed_dataset(series, window_size, batch_size, shuffle):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.map(lambda w: (w[:-1], w[-1]))
    return ds.batch(batch_size).prefetch(1)

In [10]:
# hyper parameter 정의 
WINDOW_SIZE=20
BATCH_SIZE=32

In [11]:
# trian_data는 학습용 데이터셋, test_data는 검증용 데이터셋
train_data = windowed_dataset(y_train, WINDOW_SIZE, BATCH_SIZE, True)
test_data = windowed_dataset(y_test, WINDOW_SIZE, BATCH_SIZE, False)


In [12]:
 # 모델 생성
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential([
    # 1차원 feature map 생성
    Conv1D(filters=32, kernel_size=5,
        padding="causal",
        #활성화함수를 relu로 정의
        activation="relu",
        input_shape=[WINDOW_SIZE, 1]),
    # LSTM
    LSTM(16, activation='tanh'),    
    Dense(16, activation="relu"),
    Dense(1)])

loss = Huber()
optimizer = Adam(0.005) ##3compile 시 optimizer는 adam으로 사용
model.compile(loss=Huber(), optimizer=optimizer, metrics=['mse'])

earlystopping = EarlyStopping(monitor='val_loss', patience=10)

filename = os.path.join('tmp', 'ckeckpointer.ckpt')
checkpoint = ModelCheckpoint(filename, 
                            save_weights_only=True,
                            save_best_only=True, 
                            monitor='val_loss',
                            verbose=1)

In [13]:
# 모델 학습
history = model.fit(train_data, 
                validation_data=(test_data), 
                epochs=50,   #50번 학습
                callbacks=[checkpoint, earlystopping])

Epoch 1/50
95/95 [==============================] - 15s 65ms/step - loss: 6.0876e-04 - mse: 0.0012 - val_loss: 0.0012 - val_mse: 0.0024

Epoch 00001: val_loss improved from inf to 0.00120, saving model to tmp\ckeckpointer.ckpt
Epoch 2/50
95/95 [==============================] - 4s 38ms/step - loss: 1.5345e-04 - mse: 3.0690e-04 - val_loss: 7.6799e-04 - val_mse: 0.0015

Epoch 00002: val_loss improved from 0.00120 to 0.00077, saving model to tmp\ckeckpointer.ckpt
Epoch 3/50
95/95 [==============================] - 4s 37ms/step - loss: 1.2143e-04 - mse: 2.4285e-04 - val_loss: 5.7049e-04 - val_mse: 0.0011

Epoch 00003: val_loss improved from 0.00077 to 0.00057, saving model to tmp\ckeckpointer.ckpt
Epoch 4/50
95/95 [==============================] - 4s 35ms/step - loss: 1.2326e-04 - mse: 2.4653e-04 - val_loss: 3.8464e-04 - val_mse: 7.6928e-04

Epoch 00004: val_loss improved from 0.00057 to 0.00038, saving model to tmp\ckeckpointer.ckpt
Epoch 5/50
95/95 [==============================] - 4s 

In [14]:
pred = model.predict(test_data)

In [15]:
rescaled_y_test = scaler1.inverse_transform(np.array(y_test).reshape(-1, 1))
rescaled_pred = scaler1.inverse_transform(np.array(pred).reshape(-1,1))

In [16]:
# 평가지표 함수
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

def confirm_result(y_test, y_pred):
    MAE = mean_absolute_error(y_test, y_pred)
    RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
    MSLE = mean_squared_log_error(y_test, y_pred)
    RMSLE = np.sqrt(mean_squared_log_error(y_test, y_pred))
    R2 = r2_score(y_test, y_pred)
        
    pd.options.display.float_format = '{:.5f}'.format
    Result = pd.DataFrame(data=[MAE,RMSE, RMSLE, R2],
                            index = ['MAE','RMSE', 'RMSLE', 'R2'],
                            columns=['Results'])
    return Result

In [17]:
confirm_result(rescaled_y_test[20:], rescaled_pred)

,Results
MAE,1981.30453
RMSE,2870.28895
RMSLE,0.02753
R2,0.98024
